In [ ]:
# tests out filter-types and -sizes in ChromaKey1a
#
#

import numpy as np
import datetime, time
import sys
import cv2
import matplotlib.pyplot as plt


#change to False to turn of video
showvideo=True

#load test-images and ideal results
original1 = cv2.imread("tests/testpicture1greenbg.png")
result1 = cv2.imread("tests/testpicture1result.png")

original2 = cv2.imread("tests/testpicture2greenbg.png")
result2 = cv2.imread("tests/testpicture2result.png")

original3 = cv2.imread("tests/testpicture3greenbg.png")
result3 = cv2.imread("tests/testpicture3result.png")

original4 = cv2.imread("tests/testpicture4greenbg.png")
result4 = cv2.imread("tests/testpicture4result.png")


#put functions and images into lists
imagelist=[original1,original2,original3,original4]
resultlist=[result1,result2,result3,result4]





#source:https://unsplash.com/photos/fJQSCxB18wE
alternate_background0= cv2.imread("tests/vitaliy-gavrushchenko-fJQSCxB18wE-unsplash-3.jpg")

#alternate_background= cv2.imread("white.png")



frame=cv2.imread("tests/testpicture1greenbg.png")
height,width, channels = frame.shape
#resize background to frame
#alternate_background = cv2.resize(alternate_background, (width,height))

saveframes=False
framecounter=0
distances2=np.zeros((height,width,3), np.uint8)
framenumber=0

#todo
mean=0

counter=2
counterprev=1 
#initialize array for saving current mean of all frames
meanimage=np.zeros((height,width,3), np.uint8)
framecopy=np.zeros((height,width,3))
#make copy of meanimage for calculations outside of rgb color space
meanimagecopy=np.zeros((height,width,3))
#initialize meanimage with first frame
meanimage=frame




def ChromaKeyOBS(frame4,pixel,pixel_h,gauss):
    #array that returns the result
    resultimage= np.zeros((width,height,3), np.uint8)

    #convert every pixel in frame from rgb to yuv
    yuv = cv2.cvtColor(frame4, cv2.COLOR_RGB2YUV)
    
    #green RGB(0,255,0)
    colorbase=np.array([0,200,0])
    
    #convert base color to yuv
    chroma_key=cv2.cvtColor( np.uint8([[colorbase]] ), cv2.COLOR_RGB2YUV)[0][0]
    
    
    
    #frame4[:,:,:]=np.minimum(255,2*(np.maximum(0,frame4[:,:,:]-((1/2)*chroma_key))))
    
    
    
    #make copy of chroma_key for calculations outside of uint8
    chroma_key2=np.array([float(chroma_key[0]),float(chroma_key[1]),float(chroma_key[2])])

    
    #array that saves the euclidean distance between all pixels in current frame and chroma key
    distances=np.zeros((width,height))
    
    #copy u and v chanel for calculating distance outside of uint8
    yuv_u=np.zeros((width,height))
    yuv_u[:,:]=yuv[:,:,1]
    yuv_v=np.zeros((width,height))
    yuv_v[:,:]=yuv[:,:,2]
    
    #calculate distances
    distances=((chroma_key2[1]-yuv_u)**2+(chroma_key2[2]-yuv_v)**2)**(0.5)
       

    
    #fill kernel     (kernel=np.array([[2,0,0,0,2],[0,0,1,0,0],[2,0,0,0,2]])/9)
    kernel=np.zeros((pixel,pixel_h))     
    kernel[0,0],kernel[pixel-1,0],kernel[pixel-1,pixel_h-1],kernel[0,pixel_h-1]=2/9,2/9,2/9,2/9
    kernel[int((pixel-1)/2),int((pixel_h-1)/2)]=1/9
    
    
    #apply kernel on distances
    distances=cv2.filter2D(distances,-1, kernel)
    
    if(gauss==True):
        distances=cv2.GaussianBlur(distances, (3, 3), 0)
    
   
    #assign pixels with distances<threshold to foreground and >=threshold to foreground
    threshold1=100
    threshold2=160
    distances= np.where(distances<threshold1,0,1)
    #distances= np.where(((distances-threshold1)/(threshold2/threshold1))<=0,0, np.where(((distances-threshold1)/(threshold2/threshold1))>=1,1, 3*((distances-threshold1)/(threshold2/threshold1))**2- 2*((distances-threshold1)/(threshold2/threshold1))**3))

    resultimage[:,:,0]=((distances[:,:])*frame4[:,:,0])+((1-distances[:,:])*alternate_background[:,:,0])
    resultimage[:,:,1]=((distances[:,:])*frame4[:,:,1])+((1-distances[:,:])*alternate_background[:,:,1])
    resultimage[:,:,2]=((distances[:,:])*frame4[:,:,2])+((1-distances[:,:])*alternate_background[:,:,2])
    
    return(resultimage)



def ChromaKeyOBSnofilter(frame4,gauss):
    #array that returns the result
    resultimage= np.zeros((width,height,3), np.uint8)

    #convert every pixel in frame from rgb to yuv
    yuv = cv2.cvtColor(frame4, cv2.COLOR_RGB2YUV)
    
    #green RGB(0,255,0)
    colorbase=np.array([0,200,0])
    
    #convert base color to yuv
    chroma_key=cv2.cvtColor( np.uint8([[colorbase]] ), cv2.COLOR_RGB2YUV)[0][0]
    
    
    
    #frame4[:,:,:]=np.minimum(255,2*(np.maximum(0,frame4[:,:,:]-((1/2)*chroma_key))))
    
    
    
    #make copy of chroma_key for calculations outside of uint8
    chroma_key2=np.array([float(chroma_key[0]),float(chroma_key[1]),float(chroma_key[2])])

    
    #array that saves the euclidean distance between all pixels in current frame and chroma key
    distances=np.zeros((width,height))
    
    #copy u and v chanel for calculating distance outside of uint8
    yuv_u=np.zeros((width,height))
    yuv_u[:,:]=yuv[:,:,1]
    yuv_v=np.zeros((width,height))
    yuv_v[:,:]=yuv[:,:,2]
    
    #calculate distances
    distances=((chroma_key2[1]-yuv_u)**2+(chroma_key2[2]-yuv_v)**2)**(0.5)
    
    
    if(gauss==True):
        distances=cv2.GaussianBlur(distances, (3, 3), 0)
    
    
    #assign pixels with distances<threshold to foreground and >=threshold to foreground
    threshold1=100
    threshold2=160
    distances= np.where(distances<threshold1,0,1)
    #distances= np.where(((distances-threshold1)/(threshold2/threshold1))<=0,0, np.where(((distances-threshold1)/(threshold2/threshold1))>=1,1, 3*((distances-threshold1)/(threshold2/threshold1))**2- 2*((distances-threshold1)/(threshold2/threshold1))**3))

    resultimage[:,:,0]=((distances[:,:])*frame4[:,:,0])+((1-distances[:,:])*alternate_background[:,:,0])
    resultimage[:,:,1]=((distances[:,:])*frame4[:,:,1])+((1-distances[:,:])*alternate_background[:,:,1])
    resultimage[:,:,2]=((distances[:,:])*frame4[:,:,2])+((1-distances[:,:])*alternate_background[:,:,2])
    
    return(resultimage)





def ChromaKeyOBSfullkernel(frame4,pixel,pixel_h,gauss):
    #array that returns the result
    resultimage= np.zeros((width,height,3), np.uint8)

    #convert every pixel in frame from rgb to yuv
    yuv = cv2.cvtColor(frame4, cv2.COLOR_RGB2YUV)
    
    #green RGB(0,255,0)
    colorbase=np.array([0,200,0])
    
    #convert base color to yuv
    chroma_key=cv2.cvtColor( np.uint8([[colorbase]] ), cv2.COLOR_RGB2YUV)[0][0]
    
    
    
    #frame4[:,:,:]=np.minimum(255,2*(np.maximum(0,frame4[:,:,:]-((1/2)*chroma_key))))
    
    
    
    #make copy of chroma_key for calculations outside of uint8
    chroma_key2=np.array([float(chroma_key[0]),float(chroma_key[1]),float(chroma_key[2])])

    
    #array that saves the euclidean distance between all pixels in current frame and chroma key
    distances=np.zeros((width,height))
    
    #copy u and v chanel for calculating distance outside of uint8
    yuv_u=np.zeros((width,height))
    yuv_u[:,:]=yuv[:,:,1]
    yuv_v=np.zeros((width,height))
    yuv_v[:,:]=yuv[:,:,2]
    
    #calculate distances
    distances=((chroma_key2[1]-yuv_u)**2+(chroma_key2[2]-yuv_v)**2)**(0.5)
       

    
    #fill kernel     (kernel=np.array([[2,0,0,0,2],[0,0,1,0,0],[2,0,0,0,2]])/9)
    kernel=np.zeros((pixel,pixel_h))     
    kernel[:,:]=1/(pixel*pixel_h)
    
    
    #apply kernel on distances
    distances=cv2.filter2D(distances,-1, kernel)
    
    
    
    if(gauss==True):
        distances=cv2.GaussianBlur(distances, (5, 5), -1)
    
   
    #assign pixels with distances<threshold to foreground and >=threshold to foreground
    threshold1=100
    threshold2=160
    distances= np.where(distances<threshold1,0,1)
    #distances= np.where(((distances-threshold1)/(threshold2/threshold1))<=0,0, np.where(((distances-threshold1)/(threshold2/threshold1))>=1,1, 3*((distances-threshold1)/(threshold2/threshold1))**2- 2*((distances-threshold1)/(threshold2/threshold1))**3))

    resultimage[:,:,0]=((distances[:,:])*frame4[:,:,0])+((1-distances[:,:])*alternate_background[:,:,0])
    resultimage[:,:,1]=((distances[:,:])*frame4[:,:,1])+((1-distances[:,:])*alternate_background[:,:,1])
    resultimage[:,:,2]=((distances[:,:])*frame4[:,:,2])+((1-distances[:,:])*alternate_background[:,:,2])
    
    return(resultimage)







for i in range (0, len(imagelist)):
        
    #set current input and expected output
    frame4=imagelist[i]
    result=resultlist[i]
    
    
    
    #shape of frame
    width, height, channels = frame4.shape
    
    alternate_background= alternate_background0[0:width,0:height]
    
    #make empty mask with same size as frame
    mask= np.zeros((width,height), np.uint8)
    
    #array that returns the result
    resultimage= np.zeros((width,height,3), np.uint8)
    
    #show original frame
    #cv2.imshow('original',frame4)
    
    
    result=result[950:1700,950:1400]
    plot = plt.imshow(cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
    plt.title("Ideal result")
    plt.axis('off')
    plt.show()

    plt.subplot(1, 2, 1)
    frame5=ChromaKeyOBSnofilter(frame4,False)
    frame5=frame5[950:1700,950:1400]
    plot = plt.imshow(cv2.cvtColor(frame5, cv2.COLOR_RGB2BGR))
    plt.title("(Left) ChromaKeyOBSnofilter; (Right) ChromaKeyOBSnofilter with Gaussian Blur")
    plt.axis('off')

    
    plt.subplot(1, 2, 2)
    frame5=ChromaKeyOBSnofilter(frame4,True)
    frame5=frame5[950:1700,950:1400]
    plot = plt.imshow(cv2.cvtColor(frame5, cv2.COLOR_RGB2BGR))
    plt.axis('off')
    
    plt.show()
    
    print("(1) ChromaKeyOBS; (2)  ChromaKeyOBSfullkernel; (3) ChromaKeyOBS with Gaussian Blur; (4)  ChromaKeyOBSfullkernel with Gaussian Blur")
    for x in range(1,50):
        for y in range(1,50):
            
            plt.subplot(1, 4, 1)
            
            frame5=ChromaKeyOBS(frame4,x,y,False)
            frame5=frame5[950:1700,950:1400]
            plot = plt.imshow(cv2.cvtColor(frame5, cv2.COLOR_RGB2BGR))
            plt.title("Kernel Size:("+str(x)+","+str(y)+")")
            plt.axis('off')
            
            
            plt.subplot(1, 4, 2)
            frame5=ChromaKeyOBSfullkernel(frame4,x,y,False)
            frame5=frame5[950:1700,950:1400]
            plot = plt.imshow(cv2.cvtColor(frame5, cv2.COLOR_RGB2BGR))
            plt.axis('off')
            
            
            plt.subplot(1, 4, 3)
            frame5=ChromaKeyOBS(frame4,x,y,True)
            frame5=frame5[950:1700,950:1400]
            plot = plt.imshow(cv2.cvtColor(frame5, cv2.COLOR_RGB2BGR))
            plt.axis('off')
            
            
            plt.subplot(1, 4, 4)
            frame5=ChromaKeyOBSfullkernel(frame4,x,y,True)
            frame5=frame5[950:1700,950:1400]                    
            plot = plt.imshow(cv2.cvtColor(frame5, cv2.COLOR_RGB2BGR))
            plt.axis('off')
            plt.show()
    
    
    
    
    break
    #close window if esc is pressed
    key = cv2.waitKey(1)
    if key == 27: 
        break

cv2.destroyAllWindows()




In [ ]:
# tests alpha-parameters of ChromaKeyVlahos-Method
#
#

import numpy as np
import datetime, time
import sys
import cv2
import matplotlib.pyplot as plt





#change to False to turn of video
showvideo=True

#load test-images and ideal results
original1 = cv2.imread("tests/testpicture1bluebg.png")
result1 = cv2.imread("tests/testpicture1result.png")

original2 = cv2.imread("tests/testpicture2bluebg.png")
result2 = cv2.imread("tests/testpicture2result.png")

original3 = cv2.imread("tests/testpicture3bluebg.png")
result3 = cv2.imread("tests/testpicture3result.png")

original4 = cv2.imread("tests/testpicture4bluebg.png")
result4 = cv2.imread("tests/testpicture4result.png")


#put functions and images into lists
imagelist=[original1,original2,original3,original4]
resultlist=[result1,result2,result3,result4]





#source:https://unsplash.com/photos/fJQSCxB18wE
alternate_background0= cv2.imread("tests/vitaliy-gavrushchenko-fJQSCxB18wE-unsplash-3.jpg")

#alternate_background= cv2.imread("white.png")




frame=cv2.imread("tests/testpicture1bluebg.png")
height,width, channels = frame.shape
#resize background to frame
#alternate_background = cv2.resize(alternate_background, (width,height))

saveframes=False
framecounter=0
distances2=np.zeros((height,width,3), np.uint8)
framenumber=0



def ChromaKeyVlahosv2(frame4, alpha1, alpha2, threshold):
    global framecounter
    global distances2
    
    framecounter+=1
    
    #array that returns the result
    resultimage= np.zeros((width,height,3), np.uint8)
   
    #convert every pixel in frame from rgb to yuv
    yuv = cv2.cvtColor(frame4, cv2.COLOR_RGB2YUV)
    
    frame_r=np.zeros((width,height,1))
    frame_r[:,:,0]=frame4[:,:,0]
    frame_g=np.zeros((width,height,1))
    frame_g[:,:,0]=frame4[:,:,1]
    frame_b=np.zeros((width,height,1))
    frame_b[:,:,0]=frame4[:,:,2]
    
    distances=np.zeros((width,height,1))
    distances[:,:,:]=1.0 - alpha1*((frame_b[:,:,:]) - (alpha2*(frame_g[:,:,:])))
    
    distances=np.where(distances>threshold,0,1)

    #apply mask on frame
    resultimage[:,:,0]=((distances[:,:,0])*frame4[:,:,0])+((1-distances[:,:,0])*alternate_background[:,:,0])
    resultimage[:,:,1]=((distances[:,:,0])*frame4[:,:,1])+((1-distances[:,:,0])*alternate_background[:,:,1])
    resultimage[:,:,2]=((distances[:,:,0])*frame4[:,:,2])+((1-distances[:,:,0])*alternate_background[:,:,2])

    return(resultimage)  



for i in range (0, len(imagelist)):
    
    #frame4= cv2.imread("tests/vitaliy-gavrushchenko-fJQSCxB18wE-unsplash-3.jpg")
    #set current input and expected output
    frame4=imagelist[i]
    result=resultlist[i]
    
    
    
    #shape of frame
    width, height, channels = frame4.shape
    
    alternate_background= alternate_background0[0:width,0:height]
    
    #make empty mask with same size as frame
    mask= np.zeros((width,height), np.uint8)
    
    #array that returns the result
    resultimage= np.zeros((width,height,3), np.uint8)
    
    #show original frame
    #cv2.imshow('original',frame4)
    

    
    print("(1) ChromaKeyOBS; (2)  ChromaKeyOBSfullkernel; (3) ChromaKeyOBS with Gaussian Blur; (4)  ChromaKeyOBSfullkernel with Gaussian Blur")
    for x in np.arange(0.5,255,2.5):
        for y in np.arange(0.5,255,2.5):
            
            frame5=ChromaKeyVlahosv2(frame4, 1/255,1,200)
            plot = plt.imshow(cv2.cvtColor(frame5, cv2.COLOR_RGB2BGR))
            plt.title("Kernel Size:("+str(x)+","+str(y)+")")
            plt.axis('off')
            
            plt.show()
    
    
    
    
    break
    #close window if esc is pressed
    key = cv2.waitKey(1)
    if key == 27: 
        break

cv2.destroyAllWindows()

In [ ]:
# tests parameters of 
#
#

import numpy as np
import datetime, time
import sys
import cv2
import matplotlib.pyplot as plt





#change to False to turn of video
showvideo=True

#load test-images and ideal results
original1 = cv2.imread("tests/testpicture1greenbg.png")
result1 = cv2.imread("tests/testpicture1result.png")

original2 = cv2.imread("tests/testpicture2greenbg.png")
result2 = cv2.imread("tests/testpicture2result.png")

original3 = cv2.imread("tests/testpicture3greenbg.png")
result3 = cv2.imread("tests/testpicture3result.png")

original4 = cv2.imread("tests/testpicture4greenbg.png")
result4 = cv2.imread("tests/testpicture4result.png")


#put functions and images into lists
imagelist=[original1,original2,original3,original4]
resultlist=[result1,result2,result3,result4]





#source:https://unsplash.com/photos/fJQSCxB18wE
alternate_background0= cv2.imread("tests/vitaliy-gavrushchenko-fJQSCxB18wE-unsplash-3.jpg")

#alternate_background= cv2.imread("white.png")




frame=cv2.imread("tests/testpicture1bluebg.png")
height,width, channels = frame.shape
#resize background to frame
#alternate_background = cv2.resize(alternate_background, (width,height))

saveframes=False
framecounter=0
distances2=np.zeros((height,width,3), np.uint8)
framenumber=0





def ChromaKeyVlahosv1(frame4, threshold):
    global framecounter
    global distances2
    
    framecounter+=1
    
    #array that returns the result
    resultimage= np.zeros((width,height,3), np.uint8)
   
    #convert every pixel in frame from rgb to yuv
    yuv = cv2.cvtColor(frame4, cv2.COLOR_RGB2YUV)
    
    frame_r=np.zeros((width,height,1))
    frame_r[:,:,0]=frame4[:,:,0]
    frame_g=np.zeros((width,height,1))
    frame_g[:,:,0]=frame4[:,:,1]
    frame_b=np.zeros((width,height,1))
    frame_b[:,:,0]=frame4[:,:,2]
    
    distances=np.zeros((width,height,1))
    
    distances[:,:,0]=np.abs(frame_b[:,:,0]- np.maximum(frame_g[:,:,0],frame_r[:,:,0]))
    distances=np.where(distances>threshold,0,1)

    #apply mask on frame
    resultimage[:,:,0]=((distances[:,:,0])*frame4[:,:,0])+((1-distances[:,:,0])*alternate_background[:,:,0])
    resultimage[:,:,1]=((distances[:,:,0])*frame4[:,:,1])+((1-distances[:,:,0])*alternate_background[:,:,1])
    resultimage[:,:,2]=((distances[:,:,0])*frame4[:,:,2])+((1-distances[:,:,0])*alternate_background[:,:,2])

    return(resultimage) 











for i in range (0, len(imagelist)):
    
    #frame4= cv2.imread("tests/vitaliy-gavrushchenko-fJQSCxB18wE-unsplash-3.jpg")
    #set current input and expected output
    frame4=imagelist[i]
    result=resultlist[i]
    
    
    
    #shape of frame
    width, height, channels = frame4.shape
    
    alternate_background= alternate_background0[0:width,0:height]
    
    #make empty mask with same size as frame
    mask= np.zeros((width,height), np.uint8)
    
    #array that returns the result
    resultimage= np.zeros((width,height,3), np.uint8)
    
    #show original frame
    #cv2.imshow('original',frame4)
    
    for x in np.arange(0,255):
        frame5=ChromaKeyVlahosv1(frame4,x)
        plot = plt.imshow(cv2.cvtColor(frame5, cv2.COLOR_RGB2BGR))
        dif=np.mean(abs(frame5-result))
        
        plt.title("Threshold:"+ str(x)+ ", average difference: result1:"+ str(dif))
        plt.axis('off')
            
        plt.show()
    
    
    
    
    break
    #close window if esc is pressed
    key = cv2.waitKey(1)
    if key == 27: 
        break

cv2.destroyAllWindows()